VERSAO JANEIRO 2018

In [1]:
import numpy as np
from math import *
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import *

Definicao de funcoes

In [2]:
#equacao de movimento para x2pontos
def Fx(m,x,y):
    return (-x-2*x*y)/m

#equacao de movimento para y2pontos
def Fy(m,x,y):
    return (-x**2 -y +y**2)/m

#mesmas equacoes em coordenadas polares
def Fr(m,r,th): #r2pontos
    return (-r -3*(r**2)*np.sin(th)*(np.cos(th))**2 - (r**2)*np.sin(th)**3)/m

def Fth(m,r,th): #theta2pontos
    return (r**3)*(2*np.cos(th)*(np.sin(th))**2 - np.cos(th)**3 + np.cos(th)*np.sin(th)**2)/m

#Hamiltoniana e Potencial em Coordenadas Cartesianas
def H(vx,vy,x,y):
    return 0.5*(vx**2 + vy**2 + x**2 + y**2) + (x**2)*y -(y**3)/3

def V(x,y):
    return 0.5*(x**2 + y**2) + y*(x**2) - (y**3)/3

#Hamiltoniana em Coordenadas Polares
def Hp(r,theta,vr,vth):
    return 0.5*(vr**2 + r**2) + r*vr*(np.sin(theta)*np.sin(vth) + np.cos(theta)*np.cos(vth)) + (1/2)*r**2 + (np.sin(theta)*np.cos(theta)**2 - (1/3)*np.sin(theta)**3)*r**3
#return 0.5*(vr**2 + vth**2) + 0.5*(r**2) + (r**3)*(np.sin(theta)- 4/3*np.sin**3(theta))

def Vp(r,theta): #V=V(r,theta)
    return (1/2)*r**2+(np.sin(theta)*(np.cos(theta)**2)-(1/3)*np.sin(theta)**3)*r**3

In [ ]:
#Velocidade angular, obtem-se de Hp fazendo vr = 0
"""
def vth(c,r,theta):
    if (r != 0):
        return  2*c/r -2*r*(np.sin(theta)-4/3*(np.sin(theta))**3) -1
"""

Bloco para curva das superficies equipotenciais

In [3]:
#H para K = 0, formando surperficies equipotenciais
N = 500 #quantidade de passos
h = 5/N #tamanho do passo
yl = np.linspace(-1.5,1.5,N) #valores de y para as curvas
c = [i* .4/20 for i in range(1,20)] #conjunto de valores para energia potencial

#valores de x para as curvas
def xl(yl,c):
    return np.sqrt((c + 1/3*yl**3 - 1/2*yl**2)/(1/2 + yl))

#plot das curvas
fig = plt.figure()
for hl in c:
    if (hl <= 0.16):
        plt.plot(xl(yl,hl),yl,'r-')
        plt.plot(-xl(yl,hl),yl,'r-')
    else:
        plt.plot(xl(yl,hl),yl,'k-')
        plt.plot(-xl(yl,hl),yl,'k-')
plt.axis([-1.0,1.0,-1.0,1.0])
plt.show()

C:\Users\Cliente\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: RuntimeWarning: invalid value encountered in sqrt


Bloco da definicao dos termos para o Runge-Kutta

In [4]:
#definicao de posicoes x, y e r e theta
x = list(range(N))
y = list(range(N))
r = list(range(N))
th = list(range(N))

#definicao de velocidades vx, vy e vr e vtheta
vx = list(range(N))
vy = list(range(N))
vr = list(range(N))
vth = list(range(N))

#definicao dos termos do Runge-Kutta de 4a ordem (em cartesianas)
k1vx = list(range(N))
k1vy = list(range(N))
k1x = list(range(N))
k1y = list(range(N))

k2vx = list(range(N))
k2vy = list(range(N))
k2x = list(range(N))
k2y = list(range(N))

k3vx = list(range(N))
k3vy = list(range(N))
k3x = list(range(N))
k3y = list(range(N))

k4vx = list(range(N))
k4vy = list(range(N))
k4x = list(range(N))
k4y = list(range(N))

In [5]:
a=np.linspace(-1.0,1.0,15) #para aplicar ao x
b=np.linspace(-0.5,1.0,15) #para aplicar ao y
vetPot=[]
for i in range(0,len(a)-1):
    for j in range(0,len(b)-1):
        vetPot.append(V(a[i],b[j]))
c=list(range(len(vetPot)))
d=list(range(len(vetPot)))
for k in range(0,len(c)-2):
    c[k]=vetPot[k]
for l in range(1,len(d)-1):
    d[l]=vetPot[l]
print()
print(len(c),len(d))
print(c[0:15],d[0:15])


196 196
[0.16666666666666666, 0.20452198736637511, 0.26287657920310986, 0.33927052964042759, 0.43124392614188528, 0.53633685617103977, 0.65208940719144792, 0.77604166666666663, 0.9057337220602526, 1.0387056608357628, 1.1724975704567542, 1.3046495383867833, 1.4327016520894071, 1.5541939990281826, 0.16666666666666666] [0, 0.20452198736637511, 0.26287657920310986, 0.33927052964042759, 0.43124392614188528, 0.53633685617103977, 0.65208940719144792, 0.77604166666666663, 0.9057337220602526, 1.0387056608357628, 1.1724975704567542, 1.3046495383867833, 1.4327016520894071, 1.5541939990281826, 0.16666666666666666]


Bloco da definicao das condicoes iniciais segundo o potencial

In [12]:
#definicao das condicoes iniciais de r e theta
#a=np.linspace(-1.0,1.0,15) #para aplicar ao x
#b=np.linspace(-0.5,1.0,15) #para aplicar ao y
xCond=[] #condicoes iniciais de x
yCond=[] #condicoes iniciais de y
#varredura dos elementos dos vetores de condicoes iniciais
for k in range(0,len(d)-1):
    if (d[k]<c[k]): #condicao para pontos de menor energia que o valor limite e diferentes de zero
        xCond.append(c[k])
        yCond.append(d[k])
    elif(d[k]>c[k]):
        continue
            
print(len(xCond))
print(xCond)

2
[0.16666666666666666, 194]


Bloco do Runge-Kutta. Lembre-se de que, se o plot acontecer:
No primeiro laco, apenas condicoes iniciais com 'r' diferente serao plotadas. No segundo laco,
No terceiro laco, os pontos das orbitas de todas as condicoes iniciais serao plotadas

In [13]:
#Fazendo Runge kutta nas equações de movimento para r e theta ARRUMAR COND INICIAIS A PARTIR DE H !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#d = np.linspace(0,pi,100) #condicoes iniciais genericas de theta
#f = np.linspace(0,0.4,100) #condicoes iniciais genericas de r
fig=plt.figure()
for j in range(0,len(xCond)-1): #for j in range(0,len(d)):
    x[0] = xCond[j] #r[0] = f[j]
    vx[0] = 0 #velocidade radial
    for k in range(0,len(yCond)-1): #for k in range(0,len(thCond)-1):
        y[0] = yCond[k]
        aux = abs(2*0.17 - x[0]**2 - 2*(x[0]**3)*(np.sin(y[0]) - 4/3*(np.sin(y[0]))**3))
        vy[0] = np.sqrt(aux) #velocidade angular vth[0] = np.sqrt(aux)
        for i in range(0,N-1): #for i in range(0,N-1):
            k1vx[i] = Fx(1,x[i],y[i])*h
            k1vy[i] = Fy(1,x[i],y[i])*h
            k1x[i] = vx[i]*h
            k1y[i] = vy[i]*h

            k2vx[i] = Fx(1,x[i]+k1x[i]/2,y[i]+k1y[i]/2)*h
            k2vy[i] = Fy(1,x[i]+k1x[i]/2,y[i]+k1y[i]/2)*h
            k2x[i] = (vx[i] + k1vx[i]/2)*h
            k2y[i] = (vy[i] + k1vy[i]/2)*h

            k3vx[i] = Fx(1,x[i]+k2x[i]/2,y[i]+k2y[i]/2)*h
            k3vy[i] = Fy(1,x[i]+k2x[i]/2,y[i]+k2y[i]/2)*h
            k3x[i] = (vx[i] + k2vx[i]/2)*h
            k3y[i] = (vy[i] + k2vy[i]/2)*h

            k4vx[i] = Fx(1,x[i]+k3x[i],y[i]+k3y[i])*h
            k4vy[i] = Fy(1,x[i]+k3x[i],y[i]+k3y[i])*h
            k4x[i] = (vx[i] + k3vx[i])*h
            k4y[i] = (vy[i] + k3vy[i])*h

            vx[i+1] = vx[i] + 1/6*(k1vx[i] + 2*(k2vx[i] + k3vx[i]) + k4vx[i])
            vy[i+1] = vy[i] + 1/6*(k1vy[i] + 2*(k2vy[i] + k3vy[i]) + k4vy[i])

            x[i+1] = x[i] + 1/6*(k1x[i] + 2*(k2x[i] + k3x[i]) + k4x[i])
            y[i+1] = y[i] + 1/6*(k1y[i] + 2*(k2y[i] + k3y[i]) + k4y[i])
            
            if (abs(x[i+1])>1.5 or abs(y[i+1])> 1.5):
                break
            elif (i==N):
                break

#    print(x[0],x[N-1])
#    print(r)
#    print(np.cos(th))
#    print(len(x))
#    print(type(r))
#    print(type(th))
#        if (x[i]>pow(10,-1) or y[i]>pow(10,-1)):
#            print(x[i],y[i])
#    if(x[100]> 0 and y[100] > 1):
#        plt.plot(x,y,'g-')
#    if(x[100]> sqrt(3)/2 and y[100] > 0.5):
#        plt.plot(x,y,'b-')
#    if(x[100] < sqrt(3)/2 and y[100] < 0.5):
#        plt.plot(x,y,'y-')
#    else:
#        plt.plot(x,y,'p-')
        if (x[i+1]>1.0):
            plt.plot(x[1],y[1],'bo')
        elif (x[i+1]<-1.0):
            plt.plot(x[1],y[1],'go')
        else:
            plt.plot(x[1],y[1],'yo')


plt.axis([-1.5,1.5,-1.5,1.5]) #extensao da janela do plot
plt.plot(xl(yl,0.16),yl,'r-') #plot de metade da curva equipotencial
plt.plot(-xl(yl,0.16),yl,'r-') #plot da outra metade da curva equipotencial
plt.show() #exibicao do plot

C:\Users\Cliente\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: RuntimeWarning: invalid value encountered in sqrt
